In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
digits = {
    0: [[0,0,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,0,0,0,1,0],
        [0,1,0,0,0,1,0],
        [0,1,0,0,0,1,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,0,0]],
    
    1: [[0,0,0,0,0,0,0],
        [0,0,0,1,0,0,0],
        [0,0,1,1,0,0,0],
        [0,0,0,1,0,0,0],
        [0,0,0,1,0,0,0],
        [0,0,0,1,0,0,0],
        [0,0,0,0,0,0,0]],
    
    2: [[0,0,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,1,0],
        [0,1,1,1,1,1,0],
        [0,1,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,0,0]],
    
    3: [[0,0,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,1,0],
        [0,0,1,1,1,1,0],
        [0,0,0,0,0,1,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,0,0]],
    
    4: [[0,0,0,0,0,0,0],
        [0,0,1,0,1,0,0],
        [0,0,1,0,1,0,0],
        [0,0,1,1,1,1,0],
        [0,0,0,0,1,0,0],
        [0,0,0,0,1,0,0],
        [0,0,0,0,0,0,0]],
    
    5: [[0,0,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,1,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,0,0]],
    
    6: [[0,0,0,0,0,0,0],
        [0,0,1,1,1,1,0],
        [0,1,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,0,0,0,1,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,0,0]],
    
    7: [[0,0,0,0,0,0,0],
        [0,1,1,1,1,1,0],
        [0,0,0,0,0,1,0],
        [0,0,0,0,1,0,0],
        [0,0,0,1,0,0,0],
        [0,0,1,0,0,0,0],
        [0,0,0,0,0,0,0]],
    
    8: [[0,0,1,1,1,0,0],
        [0,1,0,0,0,1,0],
        [0,1,0,0,0,1,0],
        [0,0,1,1,1,0,0],
        [0,1,0,0,0,1,0],
        [0,1,0,0,0,1,0],
        [0,0,1,1,1,0,0]],

    9: [[0,0,1,1,1,0,0],
        [0,1,0,0,0,1,0],
        [0,1,0,0,0,1,0],
        [0,0,1,1,1,1,0],
        [0,0,0,0,0,1,0],
        [0,1,0,0,0,1,0],
        [0,0,1,1,1,0,0]],
}

for i in digits:
    # https://github.com/geohot/ai-notebooks/blob/master/mnist_from_scratch.ipynb
    m = np.concatenate([np.concatenate([[x]*4 for x in y]*4) for y in digits[i]])
    digits[i] = m.reshape(28,28)

In [ ]:
# Load train and test datasets
train_df = pd.read_csv("../input/digit-recognizer/train.csv")
test_df = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
train_df.head(5)

In [ ]:
test_df.head(5)

In [ ]:
# Check if train and test dataset have the same number of columns
# Exclude first column `label` from the comparison
assert train_df.iloc[:, 1:].columns.equals(test_df.columns), "train_df and test_df do not have the same columns"

print(f"train_df.shape: {train_df.shape}")
print(f"test_df.shape: {test_df.shape}")

In [ ]:
def train_test_split(X, y, test_size=0.1):
    # Ensure that X and y are numpy arrays. If they're not, convert them.
    X = np.array(X)
    y = np.array(y)
    
    # Create a range from 0 to the length of X rows
    indices = np.arange(X.shape[0])
    # Suffle indices
    np.random.shuffle(indices)
    
    # Determine the split index
    split_idx = int(X.shape[0] * (1 - test_size))
    
    # Split the indices into training and testing parts
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    
    # Split the data and labels into training and testing sets
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Split the training dataset into training and validation datasets
y = train_df.iloc[:, 0]  # Select all rows of the first column (label column)
X = train_df.iloc[:, 1:] # Select all rows and all columns except the first one

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.1)

print(f"training dataset: {X_train.shape}")
print(f"validation dataset: {X_validate.shape}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

def build_model(nodes=[128]):
    model = nn.Sequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )

    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0)
    
    return model

In [ ]:
def get_batches_numpy(data, labels, batch_size):
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    
    num_batches = len(data) // batch_size
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx   = start_idx + batch_size
        batch     = indices[start_idx:end_idx]
        yield data[batch], labels[batch]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def get_batches_pytorch(data, labels, batch_size):
    dataset = TensorDataset(torch.tensor(data, dtype=torch.float), torch.tensor(labels))
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
def forward(inputs, labels, train=False):
    inputs = inputs if torch.is_tensor(inputs) else torch.tensor(inputs, dtype=torch.float)
    labels = labels if torch.is_tensor(labels) else torch.tensor(labels)
    
    # Forward pass with tracking gradients
    if train:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        predictions = torch.argmax(outputs, dim=1)
        loss.backward()
        optimizer.step()   
        
        return loss.item(), predictions
    
    # Forward pass without tracking gradients
    with torch.no_grad():
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        predictions = torch.argmax(outputs, dim=1)

        return loss.item(), predictions

In [ ]:
def predict(inputs):
    # Forward pass without tracking gradients
    with torch.no_grad():
        inputs = inputs if torch.is_tensor(inputs) else torch.tensor(inputs, dtype=torch.float)
        
        outputs = model(inputs)        
        probabilities = F.softmax(outputs, dim=1)
        predictions = torch.argmax(outputs, dim=1)
        
        return predictions.numpy()

In [ ]:
def accuracy(labels, predictions):
    labels = labels if torch.is_tensor(labels) else torch.tensor(labels)
    
    # Calculate the number of correct predictions
    correct_predictions = torch.sum(predictions == labels)

    # To calculate accuracy
    accuracy = correct_predictions.item() / labels.size(0)
    
    return accuracy*100

TODO:
* test multiple networks
* torch data loader
* test different learning rates
* test batchnorm
* visualize activations

In [ ]:
num_epochs=1000
batch_size=128

# Calculate number of batches
num_batches = X_train.shape[0] // batch_size

# Track losses per epoch
train_losses = [0] * num_epochs
train_accuracy = [0] * num_epochs
validate_losses = [0] * num_epochs
validate_accuracy = [0] * num_epochs

# DataLoader
# numpy: get_batches_numpy(X_train, y_train, batch_size):
dataloader = get_batches_pytorch(X_train, y_train, batch_size)

print(f"Training on {len(X_train)} samples")
print(f"Epochs: {num_epochs}")
print(f"Batches: {num_batches} with size {batch_size}")

initial_loss, _ = forward(X_train, y_train, False)
print(f"Initial loss: {initial_loss:.3f}")

print("Training started.")

for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        forward(X_batch, y_batch, True)
    
    train_loss, train_prediction = forward(X_train, y_train, False)
    train_losses[epoch] = train_loss
    train_accuracy[epoch] = accuracy(y_train, train_prediction)
    
    validate_loss, validate_prediction = forward(X_validate, y_validate, False)
    validate_losses[epoch] = validate_loss
    validate_accuracy[epoch] = accuracy(y_validate, validate_prediction)
    
    print(f"Epoch {epoch}, train loss {train_losses[epoch]:.3f}, validate loss {validate_losses[epoch]:.3f}, accuracy {validate_accuracy[epoch]:.3f}")
        
print("Training completed.")
print(f"Training loss: {train_losses[-1]:.3f}")
print(f"Validation loss: {validate_losses[-1]:.3f}")
print(f"Validation accuracy: {validate_accuracy[-1]:.3f}%")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot the results
plt.subplot(211)
plt.ylabel('Accuracy')
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(validate_accuracy, label='Validation Accuracy')
plt.legend()

plt.subplot(212)
plt.ylabel('Loss')
plt.xlabel("Epoch")
plt.plot(train_losses, label='Training Loss')
plt.plot(validate_losses, label='Validation Loss')
plt.legend()

In [ ]:
# make predictions
X_test_digits = np.array([d.flatten() for d in digits.values()])
y_pred_digits = predict(X_test_digits)

fig, ax = plt.subplots(1, 10, figsize=(10, 10))

for digit, matrix in digits.items():
    ax[digit].imshow(matrix, cmap='gray')
    ax[digit].set_title(str(y_pred_digits[digit]))
    ax[digit].axis('off')

plt.show()

In [ ]:
X_test = test_df.to_numpy()

y_test = predict(X_test)

In [ ]:
# Create submission dataframe
submission_df = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
submission_df["Label"] = y_test
submission_df.head(10)

In [ ]:
submission_df.to_csv("submission.csv", index=False)